In [40]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
spark =SparkContext(conf=conf).getOrCreate()

24/12/03 15:38:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
import os
directory= os.path.join(os.getcwd(),"data") #directory에 저장 
filename= "restaurant_reviews.csv"
filepath= os.path.join(directory, filename) #파일데이터 path로 연결 

#RDD생성 
lines= spark.textFile("file://"+filepath.replace("\\", "/"))
lines

file:///home/lab10/src/data/restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [18]:
header= lines.first()
filtered_lines= lines.filter(lambda row: row !=header)

In [19]:
filtered_lines.take(5)

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,']

In [21]:
def parse(row):
    fields=row.split(",")
    category = fields[2]
    reviews=int(fields[3])
    
    return category,reviews

In [22]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [23]:
# RDD 내 모든 row에 대해 parse()를 적용한 다음, map()추출

In [26]:
category_reviews= filtered_lines.map(parse)

In [27]:
category_reviews.take(5)

[('중식', 125), ('중식', 235), ('분식', 32), ('분식', 534), ('일식', 223)]

In [29]:
category_review_count= category_reviews.mapValues(lambda x:(x,1))
category_review_count.take(3)

[('중식', (125, 1)), ('중식', (235, 1)), ('분식', (32, 1))]

In [33]:
#실행 x
'중식' (125,1)
첫번째 중식 > (125,1) -> x
두번째 중식 > (235,1) -> y
리뷰의 갯수의 합= x[0] + y[0]
건수의 합 = x[1] + y[1]

SyntaxError: invalid syntax (2871578972.py, line 2)

In [31]:
#카테고리별 합계 

reduced_rdd= category_review_count.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
reduced_rdd.take(3)

[('중식', (360, 2)), ('분식', (566, 2)), ('일식', (287, 3))]

In [34]:
reduced_rdd.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [35]:
#평균
average= reduced_rdd.mapValues(lambda x:x[0]/x[1])

In [36]:
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [37]:
#데이터 추가 >메뉴별 합계와 평균

In [38]:
lines.take(8)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,']

In [39]:
spark.stop()